### Elementy sztucznej inteligencji
# Projekt: Binarne Drzewa Decyzyjne
autorzy:
1. Arkadiusz Florek
2. Maciej Komosa
3. Albert Pieniądz
4. Jakub Zięba

In [1]:
import numpy as np, pandas as pd, math as mat

Funkcje do obliczania entropii dla danej konkluzji

In [9]:
def prawdopodobienstwo(wartosci, konkluzje, nr_konkluzji):

    nr_konkluzji += 1
    unikatowe_wartosci = list(set(wartosci))

    wartosci = wartosci[konkluzje == nr_konkluzji]

    # tabela zawierająca częstości wystąpienia poszczególnych wartości przesłanek
    p_tab = np.zeros(len(unikatowe_wartosci))

    for indeks, unikatowa_wartosc in enumerate(unikatowe_wartosci):

        for wartosc in wartosci:

            if wartosc == unikatowa_wartosc:
                p_tab[indeks] += 1

    p_tab = p_tab / len(wartosci)
    return p_tab


In [3]:
def entropia(p_tab):
    I = 0
    for p in p_tab:
        if p == 0:
            continue
        else:
            I += -p*mat.log2(p)
    return I

Klasa węzła

In [4]:
class wezel:

    def __init__(self):
        self.value = None
        self.next = None
        self.childs = None

Importowanie danych

In [5]:
data = pd.read_csv("sport.csv")
data.head()
X = data.iloc[:, 0:-1]
Y = data.iloc[:, -1]

In [11]:
p_tab = []; p_buf = []
e_tab = []; e_buf = []

for przeslanka in range(X.shape[1]):

    for konkluzja in range(len(set(Y))):
    
        p_buf.append(prawdopodobienstwo(X.iloc[:, przeslanka].to_numpy(), Y, konkluzja))
        e_buf.append(entropia(p_buf[-1]))
    print(p_buf)
    p_tab.append(p_buf)
    e_tab.append(e_buf)
    p_buf.clear()
    e_buf.clear()

[0.34042553 0.65957447]
[0.24 0.76]
[0.37037037 0.62962963]
[0.81818182 0.18181818]
[0.8 0.2]
[0.69565217 0.30434783]
[0.63636364 0.36363636]
[0.34042553 0.42553191 0.23404255]
[0.28 0.4  0.32]
[0.2962963  0.25925926 0.44444444]
[0.36363636 0.21212121 0.42424242]
[0.32 0.44 0.24]
[0.39130435 0.30434783 0.30434783]
[0.45454545 0.         0.54545455]
[0.31914894 0.4893617  0.19148936]
[0.34 0.2  0.46]
[0.40740741 0.25925926 0.33333333]
[0.36363636 0.33333333 0.3030303 ]
[0.12 0.4  0.48]
[0.56521739 0.2173913  0.2173913 ]
[0.09090909 0.54545455 0.36363636]
[0.44680851 0.27659574 0.27659574]
[0.3  0.28 0.42]
[0.22222222 0.59259259 0.18518519]
[0.18181818 0.45454545 0.36363636]
[0.4  0.24 0.36]
[0.47826087 0.17391304 0.34782609]
[0.27272727 0.36363636 0.36363636]
[0.14893617 0.36170213 0.23404255 0.25531915]
[0.18 0.34 0.18 0.3 ]
[0.33333333 0.33333333 0.33333333 0.        ]
[0.18181818 0.18181818 0.42424242 0.21212121]
[0.32 0.   0.2  0.48]
[0.39130435 0.04347826 0.2173913  0.34782609]
[0.

[[], [], [], [], []]